In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

In [20]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images.shape # 28 * 28 이미지가 60,000개
train_labels.shape # 라벨이 60,000개
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
# CNN 모델은 '컨볼루션 계층'과 '풀링 계층'으로 구성된다.
# 이 계층들을 얼마나 많이, 또 어떠한 방식으로 쌓느냐에 따라 성능차이가 발생

# 컨볼루션 계층 => 2차원의 평면 행렬에서 지정한 영역의 값들을 하나의 값으로 압축하는 것
# 하나의 값으로 압축할 때 컨볼루션 계층은 가중치와 편향을 적용
# 풀링 계층 => 단순히 값들 중 하나를 선택해서 가져오는 방식을 취합니다.

In [21]:
# CNN은 배치(batch) 크기를 제외하고 (이미지 높이, 이미지 너비, 컬러 채널) 크기의 텐서(tensor)를 입력으로 받습니다

"""
CNN은 배치(batch) 크기를 제외하고 (이미지 높이, 이미지 너비, 컬러 채널) 크기의 텐서(tensor)를 입력으로 받습니다. MNIST 데이터는 (흑백 이미지이기 때문에) 컬러 채널(channel)이 하나지만 컬러 이미지는 (R,G,B) 세 개의 채널을 가집니다. 이 예에서는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하는 CNN을 정의하겠습니다. 이 값을 첫 번째 층의 input_shape 매개변수로 전달합니다.
"""
model = models.Sequential()
# filters     : 32    => (윈도우의 크기)
# kernel_size : (3, 3)=> 커널 사이즈
# strides=(1, 1) : 몇 칸씩 움직일 것인가
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

# pool_size=(2, 2),
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.summary()

# 위에서 Conv2D와 MaxPooling2D 층의 출력은 (높이, 너비, 채널) 크기의 3D 텐서입니다.
# 높이와 너비 차원은 네트워크가 깊어질수록 감소하는 경향을 가집니다
#  Conv2D 층에서 출력 채널의 수는 첫 번째 매개변수에 의해 결정됩니다 ((예를 들면, 32 또는 64))
# 일반적으로 높이와 너비가 줄어듦에 따라 (계산 비용 측면에서) Conv2D 층의 출력 채널을 늘릴 수 있습니다.



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


In [25]:
# 마지막에 Dense 층 추가하기
"""
모델을 완성하려면 마지막 합성곱 층의 출력 텐서(크기 (4, 4, 64))를 하나 이상의 Dense 층에 주입하여 분류를 수행합니다.
 Dense 층은 벡터(1D)를 입력으로 받는데 현재 출력은 3D 텐서입니다. 먼저 3D 출력을 1D로 펼치겠습니다. 그다음 하나 이상의 Dense 층을 그 위에 추가하겠습니다. MNIST 데이터는 10개의 클래스가 있으므로 마지막에 Dense 층에 10개의 출력과 소프트맥스 활성화 함수를 사용합니다.
"""
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

"""
output_shape의 차원은 레이어에 있는 노드의 숫자입니다.
Parameters의 숫자는 아래 공식에 각 숫자를 대입해 얻은 값입니다:
(노드의 수) * (input의 차원) + (노드의 수)

"""
# 여기에서 볼 수 있듯이 두 개의 Dense 층을 통과하기 전에 (4, 4, 64) 출력을 (1024) 크기의 벡터로 펼쳤습니다.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

In [26]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 11s 2ms/step - loss: 0.1461 - accuracy: 0.9547
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0466 - accuracy: 0.9854
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0329 - accuracy: 0.9894
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0259 - accuracy: 0.9920
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0209 - accuracy: 0.9933


In [32]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=1)


313/313 [==============================] - 0s 1ms/step - loss: 0.0296 - accuracy: 0.9914


In [33]:
print(test_acc)


0.9914000034332275
